In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## מדידת תאוצת הכובד בעזרת מטוטלת חכמה

### רקע ומוטיבציה
מדידת תאוצת הכובד $g$ היא אחד הניסויים הקלאסיים והאלגנטיים בפיזיקה.  
בעזרת מטוטלת פשוטה — חוט באורך $L$ ומשקולת בקצהו — ניתן לאמוד את $g$ מתוך מדידה של **זמן המחזור** של התנודה.  
בפרויקט זה נשתמש בנתוני חיישן (IMU) סינתטיים כדי להעריך את $g$, לבחון את תוקף הנחת הזוויות הקטנות, ולהעריך את אי־הוודאות הנובעת ממדידות חוזרות.

### המודל הפיזיקלי

#### תנודות קטנות של מטוטלת פשוטה
כאשר הזווית ההתחלתית קטנה (ביחידות רדיאנים), משוואת התנועה של המטוטלת ניתנת לקירוב ליניארי:

$$
\theta''(t) + \frac{g}{L} \theta(t) = 0
$$

הפתרון הוא תנודה הרמונית פשוטה בתדירות:

$$
\omega_0 = \sqrt{\frac{g}{L}}
$$

ולכן זמן המחזור של התנודה הוא:

$$
T = 2\pi \sqrt{\frac{L}{g}}
$$

ומכאן מתקבלת נוסחה לאמידת $g$:

$$
g = \frac{4\pi^2 L}{T^2}
$$

#### תיקון לזוויות שאינן קטנות
כאשר הזווית ההתחלתית גדולה יותר, זמן המחזור מתארך במעט. ניתן להשתמש בתיקון מסדר שני לפי פיתוח טורי של אינטגרל אליפטי:

$$
T(\theta_0) \approx T_0 \left( 1 + \frac{\theta_0^2}{16} \right)
$$

כאשר $T_0$ הוא זמן המחזור בזוויות קטנות, ו־$\theta_0$ נמדדת ברדיאנים.

#### דעיכה (חיכוך ואיבודי אנרגיה)
במערכת אמיתית קיימת דעיכה של אמפליטודת התנודה עקב חיכוך באוויר ואיבוד אנרגיה בחיבור החוט.  
האמפליטודה מתוארת לרוב על ידי דעיכה מעריכית:

$$
\theta(t) = \theta_0 e^{-\frac{t}{\tau}} \cos(\omega t)
$$

כאשר $\tau$ הוא זמן האיפוס (time constant) הקשור לגורם האיכות $Q$:

$$
Q = \pi \frac{\tau}{T}
$$

בפרויקט זה נניח כי החיכוך חלש מספיק כך שזמן המחזור נשאר כמעט קבוע, אך נוכל לבדוק זאת באמצעות ניתוח הנפילות בעוצמת הזווית לאורך זמן.

### מטרה ניסויית
1. **להעריך את תאוצת הכובד $g$** מתוך מדידות חוזרות של זמן המחזור.  
2. **לבדוק את הנחת הזוויות הקטנות** — האם זמן המחזור אכן בלתי תלוי באמפליטודה?  
3. **להעריך את אי־הוודאות** בניסוי מתוך שונות זמני המחזור בין התנודות והמדידות.

### קבצי הנתונים
- `pendulum_timeseries.csv` — נתוני עשרות תנודות עם חיישן IMU (זמן, זווית, תאוצה).  
- `trials.csv` — נתוני כל ניסוי (אורך המטוטלת, זווית שחרור, טמפרטורה, הערות).  
- `sensors.csv` — נתוני כיול החיישנים (היסט, רגישות).

### שאלת המחקר
> האם זמן המחזור של המטוטלת באמת אינו תלוי בזווית השחרור (כל עוד היא קטנה)?  
> ומהו הערך המדויק ביותר של $g$ שאנו יכולים להעריך מהנתונים הסינתטיים שלנו?


## 📊 חלק 1 — הכנת הנתונים וצעדים ראשוניים בניתוח

בשלב זה נגדיר את מבנה הנתונים בפרויקט, נטען את שלושת קבצי ה־CSV, ונבצע מיזוגים ותיקונים ראשוניים שיכינו את הנתונים לניתוח הפיזיקלי.

### מבנה הנתונים

#### 1. **pendulum_timeseries.csv**
נתוני מדידה בזמן (time-series) עבור כל ניסוי:
| עמודה | תיאור |
|:-------|:-------|
| `trial_id` | מזהה ניסוי ייחודי |
| `t` | זמן (שניות או timestamp) |
| `theta_deg` | זווית המטוטלת במעלות |
| `omega` | מהירות זוויתית (rad/s), אופציונלי |
| `sensor_ax` | תאוצה נמדדת (m/s²), אופציונלי |

#### 2. **trials.csv**
נתוני מטה-מידע על כל ניסוי:
| עמודה | תיאור |
|:-------|:-------|
| `trial_id` | מזהה ניסוי |
| `L_m` | אורך המטוטלת במטרים |
| `release_angle_deg` | זווית השחרור ההתחלתית |
| `sensor_id` | מזהה חיישן |
| `room_temp_C` | טמפרטורת חדר |
| `notes` | הערות כלליות |

#### 3. **sensors.csv**
נתוני כיול החיישנים:
| עמודה | תיאור |
|:-------|:-------|
| `sensor_id` | מזהה חיישן |
| `theta_offset_deg` | היסט בזווית המדידה |
| `gain` | גורם רגישות (כפולה/חלוקה במדידה) |

### שלבים 1–2: טעינת הנתונים ובדיקות תקינות בסיסיות

בשלב זה אנו טוענים את שלושת מקורות הנתונים לקובצי `DataFrame` של Pandas ומוודאים באופן מהיר שהקבצים נקראו כהלכה:

1. **טעינת הנתונים**
   - `pendulum_timeseries.csv` נטען כך שהעמודה `t` נשארת **עמודת שניות נומרית** (לא `datetime`). זה חשוב כי בהמשך נחשב מחזורים וקצבי דגימה מתוך שניות.
   - `trials.csv` ו־`sensors.csv` נטענים כפי שהם, וישמשו אותנו למטא־דאטה ולנתוני הכיול בהתאמה.
   - מיד לאחר הטעינה מוצגת הצצה (`head`) לשלוש הטבלאות, כדי לוודא את מבנה העמודות והערכים הראשונים.

2. **בדיקות תקינות בסיסיות (Sanity Checks)**
   - הדפסת מספר הניסויים הייחודיים (לפי `trial_id`) מתוך `trials` בודקת שאין כפילויות חריגות ברמת הניסוי.
   - הדפסת מספר החיישנים הייחודיים (לפי `sensor_id`) מתוך `sensors` בודקת שהכיולים קיימים ושמותאמים לניסויים.
   - הדפסת מספר השורות הכולל בטבלת הסדרה הזמנית (`timeseries`) נותנת אינדיקציה ראשונית להיקף הדאטה ולכך שהקריאה לקובץ הצליחה.

בדיקות אלה מספקות “תאימות בסיסית” לפני כניסה לניקוי, מיזוג וכיול — אם המספרים נראים סבירים והעמודות מופיעות כמצופה, אפשר להמשיך בבטחה לשלב הבא.


In [1]:
import pandas as pd
import numpy as np

# --- 1) Load (keep 't' as a regular numeric/float column) ---
# IMPORTANT: do NOT parse 't' as dates; it's seconds
timeseries = pd.read_csv("data/pendulum_timeseries.csv")   # ← no parse_dates=["t"]
trials     = pd.read_csv("data/trials.csv")
sensors    = pd.read_csv("data/sensors.csv")

# Preview first rows
display(timeseries.head(), trials.head(), sensors.head())

# --- 2) Basic sanity checks (lightweight) ---
print("Trials:", trials["trial_id"].nunique())
print("Sensors:", sensors["sensor_id"].nunique())
print("Total time-series samples:", len(timeseries))

,trial_id,t,theta_deg,omega,sensor_ax
0,T01,0.00,10.0993,-0.0000,0.0275
1,T01,0.01,9.9531,-0.0380,-0.0016
2,T01,0.02,10.0694,-0.0759,0.0106
3,T01,0.03,10.1819,-0.1135,-0.0006
4,T01,0.04,9.7466,-0.1508,-0.0001


,trial_id,L_m,release_angle_deg,sensor_id,room_temp_C,notes
0,T01,0.45,10,SENS_A,23.4,short pendulum small angle
1,T02,0.80,20,SENS_B,22.8,medium length higher amplitude
2,T03,1.20,15,SENS_C,24.1,long pendulum test


,sensor_id,theta_offset_deg,gain
0,SENS_A,0.8,1.02
1,SENS_B,-0.5,0.99
2,SENS_C,1.3,1.01


Trials: 3
Sensors: 3
Total time-series samples: 2840


### שלב 3: ניקוי נתוני הסדרה הזמנית (Time-Series Cleaning)

לפני שנשלב את הנתונים עם טבלאות המטא־דאטה, חשוב לוודא שטבלת המדידות (`timeseries`) נקייה,  
ממוינת, וללא כפילויות או ערכים חסרים.

1. **מיון לפי מזהה ניסוי וזמן (`trial_id`, `t`)**  
   כך נבטיח שהמדידות מוצגות בסדר כרונולוגי לכל ניסוי,  
   מה שמקל על חישוב מחזורים וזמני תנודה בהמשך.

2. **הסרת כפילויות לפי `(trial_id, t)`**  
   במידה וחלק מהמדידות מופיעות פעמיים באותו זמן,  
   נשמור רק את ההופעה הראשונה כדי למנוע טעויות חישוב.

3. **הסרת ערכים חסרים (`NaN`)**  
   נוודא שאין ערכים חסרים בעמודות הקריטיות: מזהה ניסוי, זמן, וזווית.  
   ערכים כאלה עלולים לשבש מיזוגים או ניתוחים פיזיקליים.

4. **בדיקת ייחודיות (Assertion)**  
   נוודא שאין כפילויות באינדקס (אם קיימת עמודת אינדקס),  
   כדי לוודא שכל מדידה מזוהה באופן חד־משמעי בזמן ובניסוי שלה.

לאחר ניקוי זה, הנתונים זמינים ובטוחים לשלב המיזוג עם טבלאות המטא־דאטה.


In [2]:
# --- 3) Clean time-series ---
# - sort by trial_id and t
# - drop duplicates by (trial_id, t)
# - drop obvious NaNs that break the pipeline
timeseries = (
    timeseries
    .sort_values(["trial_id", "t"], kind="mergesort")
    .dropna(subset=["trial_id", "t", "theta_deg"])
    .drop_duplicates(subset=["trial_id", "t"], keep="first")
)

# Ensure unique index
assert not timeseries.index.duplicated().any(), "Duplicate (trial_id, t) entries found!"

### שלב 4: מיזוג נתוני המדידה עם נתוני המטא־דאטה

בשלב זה נחבר בין שלושת מקורות הנתונים שלנו לטבלה מאוחדת אחת (`df`),  
כך שכל שורת מדידה תקבל גם את המידע הפיזיקלי והתפעולי הרלוונטי לה.

1. **מיזוג עם `trials.csv`**  
   כל מדידה (`timeseries`) משויכת לניסוי מסוים לפי מזהה `trial_id`.  
   באמצעות `merge(..., validate="m:1")` אנחנו מאשרים שכל ניסוי מופיע פעם אחת בלבד  
   בטבלת הניסויים — כלומר, מיזוג מסוג “רבים לאחד”.

2. **מיזוג עם `sensors.csv`**  
   לאחר מכן נוסיף את נתוני הכיול של החיישן (`sensor_id`),  
   כדי שנוכל לתקן את המדידות לפי היסט ורגישות החיישן.

3. **בדיקת הצלחת המיזוג**  
   נוודא שכל מדידה באמת קיבלה חיישן מתאים — כלומר,  
   שאין שורות שבהן `sensor_id` ריק או חסר.  
   אם יש בעיה במיפוי בין הטבלאות, תתקבל הודעת שגיאה ברורה.

בסוף שלב זה מתקבלת טבלה מאוחדת (`df`) המכילה את כל המידע הפיזיקלי הדרוש  
לכל דגימה בזמן, כולל פרטי הניסוי והחיישן.


In [3]:
# --- 4) Merge with metadata ---
df = (
    timeseries
    .merge(trials,  on="trial_id", validate="m:1")
    .merge(sensors, on="sensor_id", validate="m:1")
)

# Ensure merge success
assert df["sensor_id"].notna().all(), "Some rows are missing matching sensor_id!"


### שלב 5: תיקון כיול והכנת שדות פיזיקליים לחישוב

בשלב זה ניישם את תיקוני הכיול של החיישנים וניצור עמודות חדשות שמתאימות לעיבוד פיזיקלי:

1. **תיקון זווית המדידה**  
   כל חיישן עשוי למדוד זוויות עם היסט (offset) או רגישות שונה (gain).  
   לכן נחשב את הזווית המתוקנת לפי הנוסחה:
   
   $$
   \theta_{\text{corr}} = (\theta_{\text{raw}} - \text{offset}) \times \text{gain}
   $$

   כך נבטיח שכל המדידות מבוססות על אותה סקאלה.

2. **סינון מדידות לא פיזיקליות**  
   נוריד ערכים קיצוניים — לדוגמה, מדידות שבהן הזווית גדולה מ־$90^\circ$ או קטנה מ־$-90^\circ$,  
   מאחר והן אינן סבירות עבור מטוטלת פשוטה.

3. **המרת זווית למידות רדיאנים**  
   לצורך ניתוח פיזיקלי ונוסחאות מתמטיות נשתמש בזוויות ברדיאנים.  
   לכן ניצור עמודה חדשה `theta_rad`:
   
   $$
   \theta_{\text{rad}} = \text{deg2rad}(\theta_{\text{corr}})
   $$

לאחר שלב זה, כל הנתונים הזוויתיים במערכת מאוחדים, מתוקנים ומוכנים לניתוח.


In [4]:
# --- 5) Apply calibration and physics-friendly fields ---
# theta_corr_deg = (raw - offset) * gain
df["theta_corr_deg"] = (df["theta_deg"] - df["theta_offset_deg"]) * df["gain"]

# Drop unrealistic angles (keep physically plausible range)
df = df[df["theta_corr_deg"].between(-90, 90)]

# Radians for further math
df["theta_rad"] = np.deg2rad(df["theta_corr_deg"])


### שלב 6: בדיקות תקינות מבניות מהירות (Sanity Checks)

לפני שנמשיך לשלב הניתוח הפיזיקלי, חשוב לוודא שהנתונים המאוחדים שלנו תקינים מבנית.  
מטרת החלק הזה היא לבצע **בדיקות מהירות (assertions)** שמאשרות שהטבלה `df` נראית סבירה ועומדת בדרישות הבסיסיות.

1. **בדיקת עמודות חובה**  
   נוודא שכל העמודות הקריטיות קיימות בטבלה:
   - `trial_id` – מזהה ניסוי  
   - `t` – הזמן של כל מדידה  
   - `theta_rad` – הזווית המתוקנת ברדיאנים  
   - `L_m` – אורך המטוטלת במטרים  
   - `sensor_id` – מזהה החיישן  
   אם אחת מהעמודות חסרה, הקוד יעצור מיד עם הודעת שגיאה ברורה.

2. **בדיקת כמות הדגימות לכל ניסוי**  
   נוודא שכל ניסוי מכיל מספיק מדידות (ברירת המחדל: לפחות 50 דגימות).  
   ניסוי עם פחות דגימות עלול להיות קצר מדי ולא לאפשר ניתוח אמין של זמן המחזור.

בדיקות אלו פועלות כמו “בקרת איכות” לפני שמתקדמים –  
אם הן עוברות בהצלחה, ניתן להיות בטוחים שהנתונים מוכנים לחישוב זמני המחזור ואמידת $g$.


In [5]:
# --- 6) Minimal structural assertions (fast sanity checks) ---
# a) required columns exist
required = {"trial_id", "t", "theta_rad", "L_m", "sensor_id"}
assert required.issubset(df.columns), f"Missing expected columns: {required - set(df.columns)}"

# b) each trial has enough samples (you can tune the threshold)
sample_counts = df.groupby("trial_id").size()
assert sample_counts.ge(50).all(), f"Some trials have too few samples:\n{sample_counts}"

### שלב 7: סיכום טווח הזמן לכל ניסוי ובדיקת קצב הדגימה

בחלק זה נבצע בדיקה פשוטה שמוודאת שהנתונים הסינתטיים שלנו "נראים הגיוניים בזמן".  
כל ניסוי (trial) נמדד לאורך זמן – ולכן נרצה לראות עבור כל מזהה ניסוי:

- את זמן ההתחלה (**min**)  
- את זמן הסיום (**max**)  
- את מספר הדגימות הכולל (**count**)  
- ואת משך הניסוי (**duration = max - min**)

כך נוכל לזהות ניסויים קצרים מדי או בעלי חוסר עקביות בזמני הדגימה.  
בנוסף נחשב גם את **קצב הדגימה המשוער (sampling rate, ב-Hz)**,  
שמתקבל ממספר הדגימות חלקי משך הניסוי.

זהו צעד בדיקה בלבד, שמוודא שהנתונים מוכנים לשלבים הבאים של ניתוח המחזור ואמידת $g$.


In [6]:
# --- 7) Quick end-of-cell “time coverage” summary per trial (since t is a column) ---
coverage = (
    df.groupby("trial_id")["t"]
      .agg(["min", "max", "count"])
      .assign(duration=lambda s: s["max"] - s["min"])
)
print("\nTime coverage per trial:")
print(coverage)

# (Optional) estimate sampling rate per trial (Hz) from span and count
est_fs = df.groupby("trial_id")["t"].apply(
    lambda s: (len(s) - 1) / (s.max() - s.min()) if s.max() > s.min() else np.nan
)
print("\nEstimated sampling rate (Hz) per trial:")
print(est_fs.round(2))


Time coverage per trial:
          min      max  count  duration
trial_id                               
T01       0.0   7.9900    800    7.9900
T02       0.0  11.9875    960   11.9875
T03       0.0  17.9833   1080   17.9833

Estimated sampling rate (Hz) per trial:
trial_id
T01    100.0
T02     80.0
T03     60.0
Name: t, dtype: float64


בקובץ `df` כעת יש את כל הנתונים המאוחדים, המתוקנים והמסוננים, והוא מייצג את נקודת הסיום של שלב ההכנה.  
כל דגימה בזמן (שורה אחת בטבלה) מכילה את כל המידע הפיזיקלי, הניסויי והחיישני הדרוש לניתוח המטוטלת.

#### מבנה הנתונים:
- **עמודות מזהות:**
  - `trial_id` – מזהה הניסוי שאליו שייכת המדידה  
  - `t` – הזמן בשניות מרגע תחילת הניסוי  
- **עמודות פיזיקליות:**
  - `theta_deg` – הזווית הגולמית שנמדדה על ידי החיישן  
  - `theta_corr_deg` – הזווית המתוקנת לאחר החלת תיקון הכיול (offset + gain)  
  - `theta_rad` – הזווית המתוקנת ביחידות רדיאנים (לשימוש בנוסחאות פיזיקליות)  
- **עמודות מטא־דאטה:**
  - `L_m` – אורך המטוטלת במטרים  
  - `release_angle_deg` – זווית השחרור ההתחלתית של המטוטלת  
  - `room_temp_C` – טמפרטורת החדר במהלך הניסוי  
  - `sensor_id` – מזהה החיישן ששימש למדידה  
  - `gain`, `theta_offset_deg` – פרטי הכיול הספציפיים של אותו חיישן  
  - `notes` – הערות כלליות או תצפיות על הניסוי

בסיום שלב זה הנתונים נקיים, עקביים ומאוחדים תחת מבנה אחיד אחד (`df`),  
כך שכל ניתוח פיזיקלי בהמשך (כגון זיהוי תנודות, חישוב זמן מחזור, או אמידת תאוצת הכובד $g$)  
יכול להתבסס ישירות עליו מבלי להזדקק למיזוגים נוספים או לניקוי נוסף.

במילים אחרות — קובץ `df` הוא **בסיס הנתונים הראשי של הניסוי** ממנו נגזור את כל המדדים הבאים.


## חלק 2: הנדסת מאפיינים (Feature Engineering) – חישוב זמן מחזור והערכת תאוצת הכובד

בשלב זה נבצע את עיבוד האות הפיזיקלי מתוך נתוני הזווית של המטוטלת (`theta`)  
ונחשב עבור כל ניסוי את **זמן המחזור הממוצע**, את סטיית התקן, ואת אמידת תאוצת הכובד $g$.  
המטרה היא לעבור מהמידע הגולמי — אות תנודה בזמן — לסט מאפיינים פיזיקליים בעלי משמעות.

### השלבים העיקריים:

1. **ניקוי וסינון האות (Detrend & Smooth)**  
   נשתמש בגלגול ממוצע (`rolling mean`) כדי להסיר רעשי מדידה ולהחליק את האות,  
   כך שנוכל לזהות את נקודות החצייה של הציר (zero-crossings) בצורה אמינה.

2. **זיהוי מחזורים (Period Detection)**  
   נחשב את נקודות המעבר של הזווית מ־שלילית לחיובית (חצייה כלפי מעלה),  
   ונמדוד את ההפרש הזמני ביניהן — זהו זמן המחזור של תנודה אחת.  
   לאחר מכן נסיר חריגים בעזרת סינון סביב החציון כדי להבטיח יציבות.

3. **אגרגציה לכל ניסוי**  
   נחשב לכל ניסוי את:
   - הזמן הממוצע למחזור $\bar{T}$
   - סטיית התקן $σ_T$
   - מספר המחזורים שזוהו  

   כל אלה נשמרים בטבלה `period_stats`.

4. **חישוב תאוצת הכובד $g$**  
   לפי נוסחת המטוטלת בזוויות קטנות:
   
   $$
   g = \frac{4 \pi^2 L}{T^2}
   $$
   
   נשתמש באורך המטוטלת $L_m$ ובזמן המחזור הממוצע $\bar{T}$ כדי להעריך את $g$ לכל ניסוי.

5. **הערכת אי־ודאות וגבולות אמון (Confidence Intervals)**  
   נחשב את טעות המדידה המתקבלת משגיאות באורך המטוטלת ובמדידת המחזור,  
   נבצע הפצת אי־ודאות לפי הנגזרות החלקיות,  
   ונפיק טווח אמון של 95% סביב כל ערך $g$.

לאחר שלב זה, נוכל להשוות בין הניסויים, לבדוק את ההשפעה של אורך המטוטלת וזווית השחרור,  
ולוודא שהערכים המתקבלים של $g$ מתכנסים סביב הערך התאורטי $9.81 \text{ m/s}^2$.

### שלב 1: המרת יחידות זווית למידות רדיאנים

בשלב זה נוודא כי בעמודת הנתונים קיימים ערכי הזווית ביחידות **רדיאנים**,  
כיוון שזו היחידה הפיזיקלית הנכונה לביצוע חישובים טריגונומטריים ולניתוח תנודות.

- אם בעמודת הנתונים כבר קיימת עמודה בשם `theta_rad` — נשתמש בה כפי שהיא.  
- אם קיימת רק עמודת זוויות במעלות (`theta_deg`) — נמיר אותה לרדיאנים בעזרת הנוסחה:
  
  $$
  \theta_{\text{rad}} = \theta_{\text{deg}} \times \frac{\pi}{180}
  $$
  
- במידה ואין אף אחת מהעמודות הללו, נזרוק שגיאה שמודיעה למשתמש כי יש צורך לספק לפחות אחת מהן.

המרה זו שומרת את עמודת הזמן `t` כעמודה רגילה (לא אינדקס),  
כך שנוכל להמשיך להשתמש בה בקלות לצורך חישוב מחזורים, גרפים וסטטיסטיקות.


In [7]:
import numpy as np
import pandas as pd

# --- 0) Preconditions ---
req = {"trial_id", "t", "L_m"}
missing = req - set(df.columns)
if missing:
    raise ValueError(f"df is missing required columns: {missing}")
if not np.issubdtype(df["t"].dtype, np.number):
    raise TypeError("'t' must be numeric seconds (not datetime).")

# --- 1) Make sure radians are available (keeps 't' as a regular numeric column) ---
if "theta_rad" not in df.columns:
    if "theta_deg" not in df.columns:
        raise ValueError("Provide either 'theta_rad' or 'theta_deg'.")
    df = df.copy()
    df["theta_rad"] = np.deg2rad(df["theta_deg"])

### שלב 2: חישוב זמני מחזור לניסוי בודד (Period Detection)

בשלב זה אנו מגדירים פונקציה `periods_for_group()` שמבצעת את ניתוח האות עבור **ניסוי אחד בלבד**  
ומחזירה את זמני המחזור (הפרשי הזמן בין תנודות עוקבות).  
הפונקציה עושה שימוש ביכולות הנפוצות של Pandas בלבד — ללא ספריות חיצוניות.

#### מה עושה הפונקציה בפועל:

1. **מיון לפי זמן (`t`)**  
   כדי לוודא שהמדידות מנותחות בסדר כרונולוגי נכון.

2. **אומדן קצב הדגימה (`fs`)**  
   מחושב לפי הנוסחה:
   
   $$
   f_s = \frac{N - 1}{t_\text{max} - t_\text{min}}
   $$

   זהו קצב המדידה (מספר דגימות לשנייה) המשמש לקביעת חלון החלקה מתאים.

3. **החלקה והסרת מגמה (Detrend + Smooth)**  
   משתמשים בחלון מתגלגל (`rolling window`) שאורכו בערך 0.3 שניות:
   - תחילה מסירים מגמת רקע על ידי חיסור הממוצע המקומי.  
   - לאחר מכן מחליקים שוב כדי להפחית רעש מדידה ולהבליט את מבנה התנודות.

4. **זיהוי חציות אפס כלפי מעלה (Rising Zero-Crossings)**  
   האות עובר מ־ערך שלילי או אפס לערך חיובי — זה מציין תחילת מחזור חדש.  
   בעזרת פעולת `shift()` אנו משווים כל נקודה לנקודה הקודמת ובודקים מתי התרחש המעבר.  
   כדי לחדד את זמן החצייה המדויק, מבוצעת **אינטרפולציה ליניארית** בין שתי הדגימות שסביב המעבר.

5. **חישוב זמני המחזור**  
   נחשב את ההפרשים (`diff`) בין זמני חציות עוקבות — כל הפרש מייצג מחזור אחד של המטוטלת.

6. **סינון חריגים (Outlier Rejection)**  
   מחזורי זמן שאינם סבירים (קצרים מדי או ארוכים מדי ביחס לחציון) מוסרים:  
   רק ערכים בטווח
   
   $$
   0.5 \times \tilde{T} \leq T_i \leq 2.0 \times \tilde{T}
   $$
   
   נשמרים, כאשר $\tilde{T}$ הוא החציון של כל המחזורים.

#### תוצאת הפונקציה:
טבלת Pandas קטנה עם שתי עמודות:
- `trial_id` — מזהה הניסוי  
- `period_s` — זמני המחזור שזוהו (בשניות)

פונקציה זו היא הבסיס לחישוב הסטטיסטיקות הפיזיקליות בהמשך,  
כאשר נריץ אותה על כל ניסוי בנפרד באמצעות `groupby("trial_id").apply(...)`.


In [8]:
# --- 2) Helper to compute periods for ONE trial using only pandas ops ---
def periods_for_group(g: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame with columns [trial_id, period_s] for a single trial.
    Steps (pandas only):
      - sort by t
      - rolling mean to detrend & smooth (window ~0.3s of the local sampling rate)
      - rising zero-crossings via shift()
      - periods from diff of crossing times
      - robust outlier clip around the median
    """
    g = g.sort_values("t")
    t = g["t"]
    x = g["theta_rad"]

    # Estimate local sampling rate: (N-1) / span
    if len(t) > 1 and t.iloc[-1] > t.iloc[0]:
        fs = (len(t) - 1) / (t.iloc[-1] - t.iloc[0])
    else:
        fs = np.nan

    # Rolling window in samples (~0.3s), fallback to 21 if fs is unknown; keep odd
    w = int(max(11, round((fs * 0.3) if np.isfinite(fs) else 21)))
    if w % 2 == 0:
        w += 1

    # Detrend + smooth with rolling mean
    trend = x.rolling(w, center=True, min_periods=1).mean()
    smooth = (x - trend).rolling(w, center=True, min_periods=1).mean()

    # Rising zero-crossings (vectorized): x_{i-1} <= 0, x_i > 0
    x0, x1 = smooth.shift(1), smooth
    t0, t1 = t.shift(1), t
    m = x0.le(0) & x1.gt(0) & (x1 != x0)

    # Linear interpolation to get the exact crossing time
    alpha = (-x0) / (x1 - x0)
    t_cross = (t0 + alpha * (t1 - t0))[m]

    # Periods = diff between consecutive rising crossings
    periods = t_cross.diff().dropna()
    if periods.empty:
        return pd.DataFrame(columns=["trial_id", "period_s"])

    # Robust outlier rejection around median
    med = periods.median()
    keep = periods.between(0.5 * med, 2.0 * med)
    periods = periods[keep]

    return pd.DataFrame({"trial_id": g["trial_id"].iloc[0], "period_s": periods.to_numpy()})

### שלב 3: חישוב זמני מחזור לכל הניסויים

בשלב זה נפעיל את הפונקציה `periods_for_group()` שהוגדרה קודם  
על כל אחד מהניסויים בנפרד, בעזרת פקודת `groupby().apply()` של Pandas.

#### מה מתבצע כאן:
1. הנתונים בקובץ `df` מחולקים לפי מזהה הניסוי (`trial_id`),  
   כך שכל קבוצה מייצגת סדרת מדידות של ניסוי אחד.

2. עבור כל קבוצה כזו, מופעלת הפונקציה `periods_for_group()`  
   שמחזירה את זמני המחזור (ביחידות שניות) עבור אותו ניסוי.

3. התוצאות מכל הניסויים מאוחדות מחדש לטבלה אחת בשם `periods_df`,  
   כאשר כל שורה מציינת:
   - מזהה ניסוי (`trial_id`)  
   - זמן מחזור בודד (`period_s`)

4. בסוף התהליך אנו מבצעים `reset_index(drop=True)`  
   כדי לאפס את האינדקס ולקבל טבלה שטוחה ונקייה, נוחה לאגרגציה בהמשך.

#### התוצאה:
`periods_df` היא טבלה ארוכה (long format) שבה יש כמה עשרות או מאות מחזורים לכל ניסוי.  
על בסיס טבלה זו נחשב בהמשך ממוצעים, סטיות תקן ואומדנים לתאוצת הכובד $g$.


In [9]:
# --- 3) Compute periods for all trials (one apply) ---
periods_df = (
    df.groupby("trial_id", group_keys=False)
      .apply(periods_for_group)
      .reset_index(drop=True)
)

/tmp/ipykernel_2408/2784985075.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(periods_for_group)


### שלב 4: חישוב סטטיסטיקות זמן מחזור לכל ניסוי

לאחר שחישבנו את זמני המחזור הבודדים (`period_s`) לכל ניסוי,  
בשלב זה נבצע **אגרגציה (Aggregation)** — כלומר נחשב מדדים מסכמים לכל ניסוי.

#### חלק א' – חישוב מדדים סטטיסטיים
באמצעות `groupby("trial_id")` נרכז את כל זמני המחזור של כל ניסוי ונחשב:
- $T_{\text{mean}}$ — זמן המחזור הממוצע (בשניות)  
- $T_{\text{std}}$ — סטיית התקן של זמני המחזור (מדד לפיזור התנודות)  
- $T_{\text{count}}$ — מספר המחזורים שזוהו בניסוי  

התוצאה נשמרת בטבלה `period_stats`, שבה כל שורה מייצגת ניסוי אחד.

#### חלק ב' – חישוב קצב הדגימה המשוער (Sampling Rate)
לא חובה אך שימושי: נחשב גם את **קצב הדגימה המשוער** (Hz) לכל ניסוי, לפי הנוסחה:

$$
f_s = \frac{N - 1}{t_{\max} - t_{\min}}
$$

כאשר:
- $N$ — מספר הדגימות בניסוי  
- $t_{\max}, t_{\min}$ — זמני התחלה וסיום של המדידה  

תוצאה זו נשמרת בעמודה חדשה `fs_est`, שמתווספת לטבלה `period_stats` באמצעות `merge()`.

#### תוצאה:
הטבלה `period_stats` כוללת כעת לכל ניסוי את:
- זמן המחזור הממוצע $T_{\text{mean}}$  
- סטיית התקן $T_{\text{std}}$  
- מספר המחזורים $T_{\text{count}}$  
- קצב הדגימה המשוער $f_s$  

טבלה זו תשמש אותנו בהמשך לאמידת תאוצת הכובד $g$ ולחישוב רווחי האמון.


In [10]:
# --- 4) Aggregate period statistics per trial ---
period_stats = (
    periods_df.groupby("trial_id")["period_s"]
              .agg(T_mean="mean", T_std="std", T_count="count")
              .reset_index()
)

# (Nice-to-have) Estimated sampling rate per trial (Hz)
fs_est = (
    df.sort_values(["trial_id", "t"])
      .groupby("trial_id")["t"]
      .apply(lambda s: (s.size - 1) / (s.max() - s.min())
             if s.size > 1 and s.max() > s.min() else np.nan)
      .rename("fs_est")
      .reset_index()
)
period_stats = period_stats.merge(fs_est, on="trial_id", how="left")

### שלב 5: אומדן $g$ לכל ניסוי עם אי־ודאות (הפצת שגיאה)

בשלב זה נאמד את תאוצת הכובד $g$ לכל ניסוי על סמך מודל המטוטלת לזוויות קטנות:

$$
g \;=\; \frac{4\pi^2\,L}{T^2}
$$

כאשר:
- $L$ — אורך המטוטלת במטרים,
- $T$ — זמן המחזור. כאן נשתמש בממוצע המחזורים שנמדדו: $T_{\text{mean}}$.

#### אי־ודאות במדידות
נניח אי־ודאות באורך $L$ קבועה לכל הניסויים (אפשר לשנות לפי הצורך):
$$
\sigma_L \;=\; 0.005~\text{m}
$$

את אי־הוודאות של ממוצע הזמן נאמוד בעזרת סטיית־תקן על פי **שגיאת התקן של הממוצע**:
$$
\sigma_T \;=\; \frac{T_{\text{std}}}{\sqrt{T_{\text{count}}}}
$$
כאשר אם $T_{\text{count}}\le 1$ אין משמעות ל־$\sigma_T$ ולכן נגדיר $\sigma_T=\mathrm{NaN}$.

#### הפצת שגיאה (First-order)
נגזור את $g(L,T)$ לפי $L$ ו־$T$:

$$
\frac{\partial g}{\partial L} \;=\; \frac{4\pi^2}{T^2}, \qquad
\frac{\partial g}{\partial T} \;=\; -\,\frac{8\pi^2\,L}{T^3}.
$$

האי־ודאות המשוקללת ב־$g$ מתקבלת מחיבור ריבועי (בהנחת אי־תלות בין $L$ ל־$T$):

$$
\sigma_g \;=\; \sqrt{
\left(\frac{\partial g}{\partial L}\right)^2 \sigma_L^2 \;+\;
\left(\frac{\partial g}{\partial T}\right)^2 \sigma_T^2 }.
$$

#### רווח סמך 95%
נחשב רווח סמך בקירוב נורמלי עם $z=1.96$:

$$
g_{\text{low}} \;=\; g_{\text{est}} - 1.96\,\sigma_g, \qquad
g_{\text{high}} \;=\; g_{\text{est}} + 1.96\,\sigma_g.
$$

התוצאה (`g_stats`) כוללת לכל `trial_id` את:
- $L$, $T_{\text{mean}}$, $T_{\text{std}}$, $T_{\text{count}}$,
- אומדן $g_{\text{est}}$,
- אי־ודאות $\sigma_g$,
- ורווח סמך $[g_{\text{low}},\, g_{\text{high}}]$.

In [16]:
# --- 5) Compute g per trial with uncertainty (error propagation) ---
# Small-angle pendulum: g = 4*pi^2 * L / T^2
# Uncertainty:
#   sigma_T = T_std / sqrt(T_count)   (SE of the mean; NaN if count<=1)
#   sigma_L = default length uncertainty (set a constant or merge per-trial if known)
import numpy as np

pi = np.pi

# Collect per-trial metadata (length and release angle) from the raw df
trial_meta = (
    df.groupby("trial_id", as_index=False)
      .agg(L_m=("L_m", "first"),
           release_angle_deg=("release_angle_deg", "first"))
)

# Merge period stats with trial metadata
g_stats = period_stats.merge(trial_meta, on="trial_id", how="left")

default_sigma_L = 0.005  # meters (5 mm)
g_stats["sigma_L"] = default_sigma_L
g_stats["sigma_T"] = g_stats["T_std"] / np.sqrt(g_stats["T_count"])
g_stats.loc[g_stats["T_count"] <= 1, "sigma_T"] = np.nan  # no SE with single sample

# Gravity estimate
g_stats["g_est"] = 4 * (pi**2) * g_stats["L_m"] / (g_stats["T_mean"] ** 2)

# Error propagation (first-order)
dg_dL = 4 * (pi**2) / (g_stats["T_mean"] ** 2)
dg_dT = -8 * (pi**2) * g_stats["L_m"] / (g_stats["T_mean"] ** 3)

g_stats["sigma_g"] = np.sqrt(
    (dg_dL ** 2) * (g_stats["sigma_L"] ** 2) +
    (dg_dT ** 2) * (g_stats["sigma_T"] ** 2)
)

# 95% confidence interval
z = 1.96
g_stats["g_CI_low"]  = g_stats["g_est"] - z * g_stats["sigma_g"]
g_stats["g_CI_high"] = g_stats["g_est"] + z * g_stats["sigma_g"]

### שלב 6: הצגת התוצאות הסופיות

בשלב זה נדפיס את סיכומי התוצאות משני שלבי הניתוח — סטטיסטיקות המחזור ואומדני תאוצת הכובד —  
כדי לוודא שהחישובים הצליחו ושערכי $g$ מתקבלים בטווח מציאותי.

#### חלק א' — סטטיסטיקות זמן המחזור
הטבלה `period_stats` מציגה לכל ניסוי:
- $T_{\text{mean}}$ — זמן המחזור הממוצע (בשניות)  
- $T_{\text{std}}$ — סטיית התקן של זמני המחזור  
- $T_{\text{count}}$ — מספר המחזורים שנמצאו  
- $f_s$ — קצב הדגימה המשוער (Hz)

הנתונים הללו מאפשרים להעריך את יציבות המדידה והאיכות של האות בכל ניסוי.

#### חלק ב' — אומדן תאוצת הכובד ורווחי אמון
הטבלה `g_stats` מציגה את ערכי תאוצת הכובד שחושבו לכל ניסוי, יחד עם אי־הוודאות:

| סימון | משמעות |
|:------|:--------|
| $L_m$ | אורך המטוטלת במטרים |
| $T_{\text{mean}}$ | זמן המחזור הממוצע |
| $g_{\text{est}}$ | אומדן תאוצת הכובד לפי נוסחת המטוטלת |
| $\sigma_g$ | אי־ודאות באומדן $g$ (שגיאת התקן המשוקללת) |
| $g_{\text{CI\_low}}, g_{\text{CI\_high}}$ | גבולות רווח סמך של 95% סביב $g_{\text{est}}$ |
| $\sigma_L, \sigma_T$ | אי־ודאות במדידת האורך והזמן, בהתאמה |

בבדיקה סופית, הערכים של $g_{\text{est}}$ אמורים להיות קרובים ל־$9.8 \, \text{m/s}^2$  
ובעלי רווחי אמון צרים יחסית, מה שמעיד על ניסוי יציב ומדויק.


In [17]:
# --- 6) Done: show concise tables ---
print("\n=== Period statistics per trial ===")
print(period_stats)

print("\n=== Gravity estimates per trial (with 95% CIs) ===")
print(g_stats[[
    "trial_id", "L_m", "T_mean", "T_std", "T_count", "fs_est",
    "g_est", "sigma_g", "g_CI_low", "g_CI_high", "sigma_L", "sigma_T"
]])



=== Period statistics per trial ===
  trial_id    T_mean     T_std  T_count      fs_est
0      T01  1.342682  0.014546        5  100.000000
1      T02  1.746500  0.118054        6   80.000000
2      T03  2.198418  0.021155        7   60.000111

=== Gravity estimates per trial (with 95% CIs) ===
  trial_id   L_m    T_mean     T_std  T_count      fs_est      g_est  \
0      T01  0.45  1.342682  0.014546        5  100.000000   9.854297   
1      T02  0.80  1.746500  0.118054        6   80.000000  10.354102   
2      T03  1.20  2.198418  0.021155        7   60.000111   9.802128   

    sigma_g  g_CI_low  g_CI_high  sigma_L   sigma_T  
0  0.145279  9.569549  10.139044    0.005  0.006505  
1  0.575101  9.226903  11.481301    0.005  0.048195  
2  0.082171  9.641073   9.963183    0.005  0.007996  


## חלק 3: ניתוח (Analysis) – השוואת $g$, שינוי צורה, ובדיקת תוקף ההנחות

בשלב זה ננתח את התוצאות שחישבנו: נשווה את אומדני תאוצת הכובד $g$ בין אשכולות של אורך המטוטלת $L_m$ וזוויות שחרור, נעצב את הנתונים מחדש כדי לראות את $\bar{T}$ על פני ניסויים ואורכים, ונאמת את הנחת הזוויות הקטנות באמצעות מתאם בין זמן המחזור לאמפליטודה ההתחלתית.

### השלבים העיקריים

1. **השוואת $g$ לפי אשכולות $L_m$ וזוויות שחרור (Groupby & Binning)**  
   נגדיר מחיצות לאורך $L_m$ ולזווית השחרור $\theta_0$ (במעלות), ונחשב לכל דלי את סטטיסטיקות $g$ (מספר דגימות, ממוצע, סטיית תקן).  
   המטרה: לזהות תלות שיטתית של $g$ ב-$L_m$ או ב-$\theta_0$ שעשויה להעיד על הטיה ניסויית או חריגה מהנחת הזוויות הקטנות.

2. **שינוי צורה (Reshape): טבלת ציר של $\bar{T}$ לפי מזהה ניסוי ואורך**  
   נייצר Pivot שבו השורות הן `trial_id`, העמודות הן $L_m$, והערכים הם $\bar{T}$.  
   המטרה: סקירה מהירה של זמני המחזור הממוצעים על פני ניסויים שונים ואורכים שונים, ולאיתור דפוסים או חריגים.

3. **וולידציה: מתאם בין תקופה לאמפליטודה**  
   נחשב את מתאם פיארסון בין $\bar{T}$ לבין $\theta_0$ (זווית השחרור). בצ regime של זוויות קטנות מצפים לתלות חלשה מאוד, כלומר $|\rho|\ll 1$.  
   מתאם גדול עשוי להעיד על זוויות גדולות מדי, זיהוי מחזורים לא יציב, או רעש/הטיה בנתונים.

### תוצרים מרכזיים

- `g_by_L`, `g_by_angle` — טבלאות סיכום של $g$ לפי דליי $L_m$ ולפי דליי $\theta_0$ (עמודות: `count`, `mean`, `std`).  
- `T_pivot` — טבלת ציר של $\bar{T}$ לפי `trial_id × L_m`.  
- `corr_T_angle` — ערך מתאם פיארסון בין $\bar{T}$ ל-$\theta_0$ (או מטריצת מתאמים מלאה).

לאחר הניתוח נוכל לאשש שהאומדנים של $g$ עקביים על פני אורכים וזוויות שונות, ושבמשטר זוויות קטנות הקשר בין התקופה לאמפליטודה אכן חלש.

### שלב 1: ניתוח והשוואה באמצעות Pandas

לאחר שחישבנו את ערכי תאוצת הכובד $g$ עבור כל ניסוי, נבצע כעת ניתוח השוואתי כדי לבדוק את עקביות התוצאות ואת תוקף ההנחות הפיזיקליות.  
המטרה היא לזהות האם קיימת תלות שיטתית בין $g$ לבין **אורך המטוטלת** ($L_m$) או **זווית השחרור** ($\theta_0$), וכן לבחון את פיזור הערכים בין ניסויים.

בשלב זה נשתמש בכלים של ספריית **Pandas** לעיבוד וניתוח נתונים טבלאיים.

#### שלבי הניתוח:

1. **השוואת $g$ לפי אורכי המטוטלת וזוויות השחרור**  
   נבצע חלוקה (binning) של הנתונים לפי טווחי אורך $L_m$ ולפי טווחי זווית שחרור (במעלות).  
   עבור כל טווח נחשב את:
   - מספר הניסויים (`count`)  
   - ממוצע ערכי $g$ (`mean`)  
   - סטיית התקן (`std`)  
   
   כך נוכל לבדוק האם יש שינוי מובהק בערכי $g$ עם שינוי באורך או בזווית — דבר שאינו צפוי בתיאוריה, אלא אם יש שגיאות מדידה או חריגה מהנחת הזוויות הקטנות.

2. **הצגת תוצאות קבוצתיות**  
   נדפיס את טבלאות הסיכום:
   - `$g_{by\_L}$` — לפי קבוצות אורכים  
   - `$g_{by\_angle}$` — לפי קבוצות זוויות  
   
   טבלאות אלו מספקות מבט כולל על איכות הניסוי ועל עקביות החישובים בין סדרות ניסוי שונות.

השלב הבא יעסוק בשינוי צורת הנתונים (Pivot) ובבדיקת מתאם בין זמן המחזור לזווית השחרור, כדי לאשש את ההנחה שהתלות בין השניים חלשה עבור זוויות קטנות.



In [18]:
g_stats

,trial_id,T_mean,T_std,T_count,fs_est,L_m,release_angle_deg,sigma_L,sigma_T,g_est,sigma_g,g_CI_low,g_CI_high
0,T01,1.342682,0.014546,5,100.000000,0.45,10,0.005,0.006505,9.854297,0.145279,9.569549,10.139044
1,T02,1.746500,0.118054,6,80.000000,0.80,20,0.005,0.048195,10.354102,0.575101,9.226903,11.481301
2,T03,2.198418,0.021155,7,60.000111,1.20,15,0.005,0.007996,9.802128,0.082171,9.641073,9.963183


In [19]:
# --- Analysis with pandas: grouping, reshaping, and validation ---

import numpy as np
import pandas as pd

# Assumptions:
# - `g_stats` has one row per trial_id with columns: ['trial_id','L_m','g_est','release_angle_deg', ...]
# - `period_stats` has per-trial period statistics with columns: ['trial_id','T_mean', ...]
# If your column names differ, adjust below accordingly.

# ---------------------------
# 1) Groupby: compare g by L_m bins and by release_angle_deg buckets
# ---------------------------

# Define length bins (meters). Adjust edges as needed for your dataset.
length_bins = [0.0, 0.25, 0.50, 0.75, 1.00, 1.50, 2.00, np.inf]
length_labels = ["[0.00–0.25)", "[0.25–0.50)", "[0.50–0.75)", "[0.75–1.00)",
                 "[1.00–1.50)", "[1.50–2.00)", "≥2.00"]

g_by_L = (
    g_stats
    .assign(L_bin=pd.cut(g_stats["L_m"], bins=length_bins, labels=length_labels, right=False))
    .dropna(subset=["L_bin", "g_est"])
    .groupby("L_bin", observed=True)["g_est"]
    .agg(count="count", mean="mean", std="std")
    .reset_index()
    .sort_values("L_bin")
)

# Define angle buckets (degrees). Small-angle regime typically ≤ ~15°; include wider buckets for diagnostics.
angle_bins = [0.0, 5.0, 10.0, 15.0, 20.0, 30.0, 45.0, np.inf]
angle_labels = ["0–5°", "5–10°", "10–15°", "15–20°", "20–30°", "30–45°", ">45°"]

g_by_angle = (
    g_stats
    .assign(angle_bucket=pd.cut(g_stats["release_angle_deg"], bins=angle_bins, labels=angle_labels, right=False))
    .dropna(subset=["angle_bucket", "g_est"])
    .groupby("angle_bucket", observed=True)["g_est"]
    .agg(count="count", mean="mean", std="std")
    .reset_index()
    .sort_values("angle_bucket")
)

print("=== g by L_m bins ===")
print(g_by_L.to_string(index=False))

print("\n=== g by release_angle_deg buckets ===")
print(g_by_angle.to_string(index=False))


=== g by L_m bins ===
      L_bin  count      mean  std
[0.25–0.50)      1  9.854297  NaN
[0.75–1.00)      1 10.354102  NaN
[1.00–1.50)      1  9.802128  NaN

=== g by release_angle_deg buckets ===
angle_bucket  count      mean  std
      10–15°      1  9.854297  NaN
      15–20°      1  9.802128  NaN
      20–30°      1 10.354102  NaN


#### שלב 2: שינוי צורה – טבלת ציר של $\bar{T}$ לפי ניסוי ואורך המטוטלת

בשלב זה נרצה להציג את זמני המחזור הממוצעים ($\bar{T}$) של כל ניסוי בצורה נוחה להשוואה חזותית.

נשלב תחילה את טבלת הסטטיסטיקות `period_stats` עם עמודת אורכי המטוטלת `L_m` מתוך `g_stats`, כדי להבטיח שלכל מזהה ניסוי (`trial_id`) קיים גם ערך אורך מתאים.  
לאחר מכן ניצור **טבלת ציר (Pivot Table)** שבה:

- **השורות** מייצגות את מזהי הניסויים (`trial_id`),  
- **העמודות** מייצגות את אורכי המטוטלת ($L_m$),  
- **התאים** מכילים את הערכים של זמן המחזור הממוצע ($\bar{T}$).

אם לכל ניסוי יש רק אורך אחד, רוב הערכים בטבלה יהיו ריקים, אך עדיין מתקבלת תצוגה נוחה שמאפשרת:
- לזהות במהירות את הקשר בין זמן המחזור לאורך,  
- לאתר ניסויים חריגים או מדידות חסרות,  
- להבחין בדפוסים כלליים בין קבוצות ניסויים.

טבלה זו (`T_pivot`) מהווה שלב ביניים שימושי לפני ביצוע ניתוחים מתקדמים יותר — כגון מציאת מתאם בין תקופה לאמפליטודה או בדיקת עקביות בין סדרות ניסוי שונות.


In [20]:
# ---------------------------
# 2) Reshape: pivot table of T_mean by trial_id × L_m
# ---------------------------
# Join L_m onto period_stats to ensure each trial's length is available.
trial_lengths = g_stats[["trial_id", "L_m"]].drop_duplicates()

period_with_L = (
    period_stats[["trial_id", "T_mean"]]
    .merge(trial_lengths, on="trial_id", how="left")
)

# Create a pivot: rows = trial_id, columns = L_m, values = T_mean.
# If each trial has a single L_m, this will be a mostly diagonal-like layout;
# still useful to quickly scan T_mean aligned by length values.
T_pivot = pd.pivot_table(
    period_with_L,
    index="trial_id",
    columns="L_m",
    values="T_mean",
    aggfunc="first"
)

print("\n=== Pivot table: T_mean by trial_id × L_m ===")
print(T_pivot)


=== Pivot table: T_mean by trial_id × L_m ===
L_m           0.45    0.80      1.20
trial_id                            
T01       1.342682     NaN       NaN
T02            NaN  1.7465       NaN
T03            NaN     NaN  2.198418


#### שלב 3: ולידציה – בדיקת הקשר בין זמן המחזור לזווית השחרור

לפי התיאוריה של המטוטלת הקלאסית, כאשר זווית התנודה ההתחלתית קטנה (בדרך כלל עד כ־$15^\circ$),  
זמן המחזור $T$ כמעט **שאינו תלוי באמפליטודה**.  
לכן נצפה לראות מתאם חלש מאוד בין זמן המחזור הממוצע $\bar{T}$ לבין זווית השחרור $\theta_0$.

בשלב זה נחשב את **מתאם פיארסון** בין שני המשתנים:
- $\bar{T}$ — זמן המחזור הממוצע מכל ניסוי  
- $\theta_0$ — זווית השחרור (במעלות)

נשתמש ב־`pandas.DataFrame.corr(method='pearson')` כדי למדוד את עוצמת הקשר הליניארי ביניהם.

תוצאות אפשריות:
- אם $|r| \ll 1$ (למשל $r \approx 0.0$–$0.2$) → הקשר חלש מאוד, בהתאם להנחת הזוויות הקטנות.  
- אם $r$ גבוה יותר (למשל $r > 0.4$) → ייתכן שהניסויים כללו זוויות גדולות מדי,  
  או שהמדידה של זמני המחזור אינה מדויקת מספיק.

בדיקה זו מהווה מבחן עקביות חשוב לתוקף הפיזיקלי של הנתונים ולנכונות ההנחות על שיטת המדידה.


In [21]:
# ---------------------------
# 3) Validation: correlation between period and amplitude
# ---------------------------
# Expectation: at small angles, the period should be (nearly) independent of amplitude (weak correlation).
# We'll compute Pearson correlation between per-trial T_mean and release_angle_deg.

period_angle = (
    period_stats[["trial_id", "T_mean"]]
    .merge(g_stats[["trial_id", "release_angle_deg"]], on="trial_id", how="left")
    .dropna(subset=["T_mean", "release_angle_deg"])
)

if not period_angle.empty:
    corr_matrix = period_angle[["T_mean", "release_angle_deg"]].corr(method="pearson")
    corr_T_angle = corr_matrix.loc["T_mean", "release_angle_deg"]
    print("\n=== Validation: Pearson correlation (T_mean vs. release_angle_deg) ===")
    print(f"r = {corr_T_angle:.3f}  (|r| << 1 suggests weak dependence at small angles)")
else:
    print("\n=== Validation: Not enough data to compute correlation (missing T_mean or release_angle_deg). ===")



=== Validation: Pearson correlation (T_mean vs. release_angle_deg) ===
r = 0.472  (|r| << 1 suggests weak dependence at small angles)
